# Qurrium-Qua_libs paerser


## Qua_libs

It returns single-shot result.

```py
results = job.result()
"""
list[tuple[str, np.ndarray[int, np.dtype[int]]]]:
    List of tuples containing the bitstring and the corresponding gate indices.

.. code-block:: txt
    # For examplem, a 3-qubit system
    [("010", [0, 1, 2]), ("110", [2, 1, 0]), ...]
"""
```

```py
ideal_results = job.ideal_result()
"""
list[tuple[dict[str, float], np.ndarray[int, np.dtype[int]]]]:
    List of tuples containing the probabilities and the corresponding gate indices.

.. code-block:: txt
    # For examplem, a 3-qubit system
    [(
        {"010": 0.5,"110": 0.5}, [0, 1, 2]
    ), (
        {"110": 0.5, "010": 0.5}, [0, 1, 2]
    ), ...]
"""
```


In [25]:
from typing import Optional
from pathlib import Path

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

from qurry import ShadowUnveil
from qurry.qurrent.classical_shadow import ShadowUnveilExperiment
from qurry.qurrent.classical_shadow.arguments import ShadowUnveilArguments
from qurry.qurrium.experiment.utils import exp_id_process
from qurry.tools import current_time

## Circuits


In [26]:
def generate_1_qubits_delay_circuit(delay: int) -> QuantumCircuit:
    """
    Generate a quantum circuit with a delay applied to each qubit.

    Args:
        delay (int): Delay to apply to each qubit.

    Returns:
        QuantumCircuit: The generated quantum circuit.
    """
    qc = QuantumCircuit(1)
    qc.x(0)
    qc.delay(delay, 0)
    return qc

In [27]:
def generate_n_qubits_delay_circuit(n: int, delay: int) -> QuantumCircuit:
    """
    Generate a quantum circuit with a delay applied to each qubit.

    Args:
        n (int): Number of qubits in the circuit.
        delay (int): Delay to apply to each qubit.

    Returns:
        QuantumCircuit: The generated quantum circuit.
    """
    qc = QuantumCircuit(n)
    for i in range(n):
        qc.x(i)
        qc.delay(delay, i)
    return qc

In [28]:
q1_delay = generate_1_qubits_delay_circuit(20)
q1_delay.draw()

┌───┐┌───────────────┐
q: ┤ X ├┤ Delay(20[dt]) ├
   └───┘└───────────────┘

In [29]:
q3_delay = generate_n_qubits_delay_circuit(3, 20)
q3_delay.draw()

┌───┐┌───────────────┐
q_0: ┤ X ├┤ Delay(20[dt]) ├
     ├───┤├───────────────┤
q_1: ┤ X ├┤ Delay(20[dt]) ├
     ├───┤├───────────────┤
q_2: ┤ X ├┤ Delay(20[dt]) ├
     └───┘└───────────────┘

## Qurrium to Qua_libs


In [30]:
exp_method04 = ShadowUnveil()

In [31]:
exp_method04.add(q1_delay, "q1_delay")
exp_method04.add(q3_delay, "q3_delay")

'q3_delay'

In [32]:
exp_q1_delay_01 = exp_method04.measure("q1_delay", times=1000, shots=1)

/home/nccu/uv-python-venv/quantumsphere12/lib/python3.12/site-packages/qurry/qurrium/utils/inputfixer.py:184: QurryUnrecongnizedArguments: "| The following keys are not recognized as arguments for main process of experiment, but still kept in experiment record. Similar: ['files' maybe '['times']']. Unknown: ['filename']."
  warnings.warn(
/usr/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [33]:
exp_q3_delay_01 = exp_method04.measure("q3_delay", times=1000, shots=1)

/home/nccu/uv-python-venv/quantumsphere12/lib/python3.12/site-packages/qurry/qurrium/utils/inputfixer.py:184: QurryUnrecongnizedArguments: "| The following keys are not recognized as arguments for main process of experiment, but still kept in experiment record. Similar: ['files' maybe '['times']']. Unknown: ['filename']."
  warnings.warn(
/usr/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.12/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [34]:
def check_classical_shadow_exp(
    classical_shadow_exp: ShadowUnveilExperiment,
) -> tuple[dict[int, dict[int, int]], dict[int, int]]:
    """
    Check if the classical shadow experiment is valid for conversion.

    Args:
        classical_shadow_exp (ShadowUnveilExperiment):
            The Qurry experiment to check.

    Raises:
        TypeError: If the input is not a ShadowUnveilExperiment.
        ValueError: If required attributes are missing or invalid.

    Returns:
        tuple[dict[int, dict[int, int]], dict[int, int]]:
            random_unitary_ids mapping from the experiment and registers mapping.
    """
    if not isinstance(classical_shadow_exp, ShadowUnveilExperiment):
        raise TypeError(
            "The input must be an instance of ShadowUnveilExperiment "
            "from qurry.qurrent.classical_shadow."
        )
    if classical_shadow_exp.args.unitary_located is None:
        raise ValueError("The unitary located must be specified in the experiment.")
    if classical_shadow_exp.args.qubits_measured is None:
        raise ValueError("The qubits measured must be specified in the experiment.")
    if classical_shadow_exp.args.registers_mapping is None:
        raise ValueError("The registers mapping must be specified in the experiment.")
    if any(
        qi not in classical_shadow_exp.args.unitary_located
        for qi in classical_shadow_exp.args.qubits_measured
    ):
        raise ValueError("All measured qubits must be part of the unitary located.")
    if classical_shadow_exp.commons.shots != 1:
        raise ValueError("The number of shots must be 1 for QuaLibs conversion.")
    if "random_unitary_ids" not in classical_shadow_exp.beforewards.side_product:
        raise ValueError(
            "The experiment must have 'random_unitary_ids' in side products."
        )

    random_unitary_ids: dict[int, dict[int, int]] = (
        classical_shadow_exp.beforewards.side_product["random_unitary_ids"]
    )
    return random_unitary_ids, classical_shadow_exp.args.registers_mapping

In [35]:
def get_gate_indices(
    random_unitary_ids: dict[int, int],
    registers_mapping: dict[int, int],
) -> list[int]:
    """
    Get the gate indices for the QuaLibs format.

    Args:
        random_unitary_ids (dict[int, int]):
            Mapping of qubit indices to random unitary.
        registers_mapping (dict[int, int]):
            Mapping of qubit indices and classical registers.
    Returns:
        list[int]: A list of gate indices corresponding to the qubits.
    """
    return [random_unitary_ids[qi] for qi in registers_mapping.keys()]


def single_shots_processing(
    idx: int,
    single_shot_counts: dict[str, int],
    random_unitary_ids: dict[int, int],
    registers_mapping: dict[int, int],
) -> tuple[str, list[int]]:
    """Single shot processing for QuaLibs format.
    Args:
        idx (int):
            The index of the single shot counts and random unitary.
        single_shot_counts (dict[str, int]):
            The counts of single-shot results.
        random_unitary_ids (dict[int, int]):
            Mapping of qubit indices to random unitary.
        registers_mapping (dict[int, int]):
            Mapping of qubit indices and classical registers.

    Returns:
        tuple[str, list[int]]:
            A tuple containing the bitstring and the corresponding gate indices.
    """

    if len(single_shot_counts) != 1:
        raise ValueError(
            "There should be exactly one key in single_shot_counts."
            + f" Found: {len(single_shot_counts)} keys in index {idx}."
        )
    only_bitstring = list(single_shot_counts.keys())[0]
    return (
        only_bitstring[: len(registers_mapping)],
        get_gate_indices(random_unitary_ids, registers_mapping),
    )


def qurrium_single_shot_to_qua_libs_result(
    classical_shadow_exp: ShadowUnveilExperiment,
) -> list[tuple[str, list[int]]]:
    """
    Convert a Qurrium single-shot experiment to QuaLibs format result.

    Args:
        qurry_exp (ShadowUnveil): The Qurry experiment to convert.

    Returns:
        list[tuple[str, list[int]]]:
            List of tuples containing the bitstring and the corresponding gate indices.
    """
    random_unitary_ids, registers_mapping = check_classical_shadow_exp(
        classical_shadow_exp
    )

    result = [
        single_shots_processing(
            idx, single_counts, single_random_unitary_id, registers_mapping
        )
        for (idx, single_random_unitary_id), single_counts in zip(
            random_unitary_ids.items(), classical_shadow_exp.afterwards.counts
        )
    ]
    return result

In [36]:
qurrium_single_shot_to_qua_libs_result(exp_method04.exps[exp_q1_delay_01])

[('1', [0]),
 ('0', [0]),
 ('0', [1]),
 ('0', [1]),
 ('1', [1]),
 ('1', [2]),
 ('1', [0]),
 ('0', [1]),
 ('1', [0]),
 ('1', [1]),
 ('1', [0]),
 ('1', [2]),
 ('1', [0]),
 ('1', [1]),
 ('0', [0]),
 ('0', [1]),
 ('1', [0]),
 ('1', [2]),
 ('0', [0]),
 ('1', [2]),
 ('0', [1]),
 ('1', [1]),
 ('1', [2]),
 ('0', [0]),
 ('1', [1]),
 ('1', [2]),
 ('0', [0]),
 ('1', [2]),
 ('0', [1]),
 ('1', [1]),
 ('1', [2]),
 ('1', [2]),
 ('0', [1]),
 ('0', [0]),
 ('0', [1]),
 ('0', [1]),
 ('1', [2]),
 ('0', [0]),
 ('1', [0]),
 ('1', [1]),
 ('0', [1]),
 ('0', [1]),
 ('0', [0]),
 ('1', [2]),
 ('0', [0]),
 ('1', [2]),
 ('1', [2]),
 ('1', [1]),
 ('0', [0]),
 ('1', [1]),
 ('1', [1]),
 ('1', [1]),
 ('0', [1]),
 ('0', [1]),
 ('1', [2]),
 ('1', [1]),
 ('0', [0]),
 ('1', [2]),
 ('0', [1]),
 ('1', [0]),
 ('1', [2]),
 ('1', [2]),
 ('0', [1]),
 ('1', [0]),
 ('1', [0]),
 ('0', [0]),
 ('1', [2]),
 ('1', [1]),
 ('0', [1]),
 ('1', [0]),
 ('1', [2]),
 ('0', [0]),
 ('0', [0]),
 ('1', [2]),
 ('1', [2]),
 ('1', [2]),
 ('0', [1]),

In [37]:
qurrium_single_shot_to_qua_libs_result(exp_method04.exps[exp_q3_delay_01])

[('110', [0, 2, 2]),
 ('111', [1, 1, 1]),
 ('110', [0, 0, 0]),
 ('101', [2, 1, 2]),
 ('111', [1, 1, 0]),
 ('011', [2, 2, 0]),
 ('011', [2, 0, 1]),
 ('101', [0, 0, 2]),
 ('011', [0, 1, 0]),
 ('110', [0, 1, 2]),
 ('110', [1, 0, 2]),
 ('010', [1, 2, 0]),
 ('011', [2, 1, 1]),
 ('011', [2, 1, 1]),
 ('110', [0, 2, 2]),
 ('110', [0, 2, 2]),
 ('011', [2, 1, 1]),
 ('011', [1, 1, 0]),
 ('001', [2, 1, 0]),
 ('111', [2, 0, 1]),
 ('111', [2, 1, 2]),
 ('111', [2, 1, 1]),
 ('100', [1, 1, 2]),
 ('111', [2, 0, 2]),
 ('000', [1, 0, 0]),
 ('011', [0, 0, 1]),
 ('101', [2, 1, 2]),
 ('110', [1, 2, 1]),
 ('011', [1, 1, 0]),
 ('111', [2, 0, 2]),
 ('111', [0, 1, 2]),
 ('110', [1, 0, 2]),
 ('110', [0, 1, 2]),
 ('101', [1, 0, 2]),
 ('111', [1, 2, 1]),
 ('001', [0, 0, 0]),
 ('101', [2, 1, 0]),
 ('011', [0, 1, 1]),
 ('010', [0, 1, 1]),
 ('111', [2, 1, 0]),
 ('110', [1, 0, 2]),
 ('011', [2, 1, 0]),
 ('111', [0, 2, 2]),
 ('100', [0, 0, 2]),
 ('111', [2, 2, 2]),
 ('111', [0, 2, 0]),
 ('101', [0, 0, 2]),
 ('001', [2, 

## Statevector


In [38]:
def single_shots_processing_ideal(
    idx: int,
    circuit: QuantumCircuit,
    random_unitary_ids: dict[int, int],
    registers_mapping: dict[int, int],
) -> tuple[dict[str, float], list[int]]:
    """Single shot processing for QuaLibs format.
    Args:
        idx (int):
            The index of the single shot counts and random unitary.
        circuit (QuantumCircuit):
            The quantum circuit to process.
        random_unitary_ids (dict[int, int]):
            Mapping of qubit indices to random unitary.
        registers_mapping (dict[int, int]):
            Mapping of qubit indices and classical registers.

    Returns:
        tuple[dict[str, float], list[int]]:
            A tuple containing the bitstring and the corresponding gate indices.
    """
    no_measiure_circ = circuit.remove_final_measurements(inplace=False)
    assert (
        no_measiure_circ is not None
    ), f"Circuit must have measurements removed. Index: {idx}"
    state = Statevector(no_measiure_circ)
    probs: dict[str, float] = state.probabilities_dict()

    return probs, get_gate_indices(random_unitary_ids, registers_mapping)


def qurrium_single_shot_to_qua_libs_ideal_result(
    classical_shadow_exp: ShadowUnveilExperiment,
) -> list[tuple[dict[str, float], list[int]]]:
    """
    Convert a Qurrium single-shot experiment to QuaLibs format ideal result.

    Args:
        classical_shadow_exp (ShadowUnveil): The Qurry experiment to convert.

    Returns:
        list[tuple[dict[str, float], list[int]]]:
            List of tuples containing the probabilities and the corresponding gate indices.
    """
    random_unitary_ids, registers_mapping = check_classical_shadow_exp(
        classical_shadow_exp
    )

    ideal_result = [
        single_shots_processing_ideal(
            idx, circuit, single_random_unitary_id, registers_mapping
        )
        for (idx, single_random_unitary_id), circuit in zip(
            random_unitary_ids.items(),
            classical_shadow_exp.beforewards.circuit,
        )
    ]
    return ideal_result

In [39]:
bla01 = qurrium_single_shot_to_qua_libs_ideal_result(exp_method04.exps[exp_q1_delay_01])
bla01[:10]

[({np.str_('0'): np.float64(0.4999999999999999),
   np.str_('1'): np.float64(0.5000000000000001)},
  [0]),
 ({np.str_('0'): np.float64(0.4999999999999999),
   np.str_('1'): np.float64(0.5000000000000001)},
  [0]),
 ({np.str_('0'): np.float64(0.4999999999999999),
   np.str_('1'): np.float64(0.5000000000000001)},
  [1]),
 ({np.str_('0'): np.float64(0.4999999999999999),
   np.str_('1'): np.float64(0.5000000000000001)},
  [1]),
 ({np.str_('0'): np.float64(0.4999999999999999),
   np.str_('1'): np.float64(0.5000000000000001)},
  [1]),
 ({np.str_('1'): np.float64(1.0)}, [2]),
 ({np.str_('0'): np.float64(0.4999999999999999),
   np.str_('1'): np.float64(0.5000000000000001)},
  [0]),
 ({np.str_('0'): np.float64(0.4999999999999999),
   np.str_('1'): np.float64(0.5000000000000001)},
  [1]),
 ({np.str_('0'): np.float64(0.4999999999999999),
   np.str_('1'): np.float64(0.5000000000000001)},
  [0]),
 ({np.str_('0'): np.float64(0.4999999999999999),
   np.str_('1'): np.float64(0.5000000000000001)},
  [1

## Qua_libs to Qurrium


In [40]:
qua_result_01 = qurrium_single_shot_to_qua_libs_result(
    exp_method04.exps[exp_q1_delay_01]
)
qua_result_01

[('1', [0]),
 ('0', [0]),
 ('0', [1]),
 ('0', [1]),
 ('1', [1]),
 ('1', [2]),
 ('1', [0]),
 ('0', [1]),
 ('1', [0]),
 ('1', [1]),
 ('1', [0]),
 ('1', [2]),
 ('1', [0]),
 ('1', [1]),
 ('0', [0]),
 ('0', [1]),
 ('1', [0]),
 ('1', [2]),
 ('0', [0]),
 ('1', [2]),
 ('0', [1]),
 ('1', [1]),
 ('1', [2]),
 ('0', [0]),
 ('1', [1]),
 ('1', [2]),
 ('0', [0]),
 ('1', [2]),
 ('0', [1]),
 ('1', [1]),
 ('1', [2]),
 ('1', [2]),
 ('0', [1]),
 ('0', [0]),
 ('0', [1]),
 ('0', [1]),
 ('1', [2]),
 ('0', [0]),
 ('1', [0]),
 ('1', [1]),
 ('0', [1]),
 ('0', [1]),
 ('0', [0]),
 ('1', [2]),
 ('0', [0]),
 ('1', [2]),
 ('1', [2]),
 ('1', [1]),
 ('0', [0]),
 ('1', [1]),
 ('1', [1]),
 ('1', [1]),
 ('0', [1]),
 ('0', [1]),
 ('1', [2]),
 ('1', [1]),
 ('0', [0]),
 ('1', [2]),
 ('0', [1]),
 ('1', [0]),
 ('1', [2]),
 ('1', [2]),
 ('0', [1]),
 ('1', [0]),
 ('1', [0]),
 ('0', [0]),
 ('1', [2]),
 ('1', [1]),
 ('0', [1]),
 ('1', [0]),
 ('1', [2]),
 ('0', [0]),
 ('0', [0]),
 ('1', [2]),
 ('1', [2]),
 ('1', [2]),
 ('0', [1]),

In [41]:
qua_ideal_result_01 = qurrium_single_shot_to_qua_libs_ideal_result(
    exp_method04.exps[exp_q1_delay_01]
)
qua_ideal_result_01

[({np.str_('0'): np.float64(0.4999999999999999),
   np.str_('1'): np.float64(0.5000000000000001)},
  [0]),
 ({np.str_('0'): np.float64(0.4999999999999999),
   np.str_('1'): np.float64(0.5000000000000001)},
  [0]),
 ({np.str_('0'): np.float64(0.4999999999999999),
   np.str_('1'): np.float64(0.5000000000000001)},
  [1]),
 ({np.str_('0'): np.float64(0.4999999999999999),
   np.str_('1'): np.float64(0.5000000000000001)},
  [1]),
 ({np.str_('0'): np.float64(0.4999999999999999),
   np.str_('1'): np.float64(0.5000000000000001)},
  [1]),
 ({np.str_('1'): np.float64(1.0)}, [2]),
 ({np.str_('0'): np.float64(0.4999999999999999),
   np.str_('1'): np.float64(0.5000000000000001)},
  [0]),
 ({np.str_('0'): np.float64(0.4999999999999999),
   np.str_('1'): np.float64(0.5000000000000001)},
  [1]),
 ({np.str_('0'): np.float64(0.4999999999999999),
   np.str_('1'): np.float64(0.5000000000000001)},
  [0]),
 ({np.str_('0'): np.float64(0.4999999999999999),
   np.str_('1'): np.float64(0.5000000000000001)},
  [1

In [ ]:
# qua_libs to qurrium transformation
def qua_libs_result_to_qurrium_single_shot(
    result: list[tuple[str, list[int]]],
    exp_name: str = "experiment.qua_libs",
    tags: Optional[tuple[str, ...]] = None,
    save_location: Optional[str] = None,
) -> ShadowUnveilExperiment:
    """
    Convert QuaLibs result to Qurrium single-shot experiment format.
    We assume that the number of qubits is equal to the number of classical registers.
    And their mapping is identity.

    Args:
        result (list[tuple[str, list[int]]]): The QuaLibs result to convert.
        exp_name (str): The name of the experiment.
        tags (Optional[tuple[str, ...]]): Tags for the experiment.
        save_location (Optional[str]): The location to save the experiment.

    Returns:
        ShadowUnveilExperiment: The converted Qurrium single-shot experiment.
    """
    if tags is None:
        tags = ()
    elif not isinstance(tags, tuple):
        raise TypeError("Tags must be a tuple of strings.")

    sample_bitstring, sample_unitary_ids = result[0]
    if not isinstance(sample_bitstring, str):
        raise ValueError("The first element of the result must be a bitstring (str).")

    num_classical_register = len(sample_bitstring)
    num_random_unitary = len(sample_unitary_ids)

    args = {
        "exp_name": exp_name,
        "times": len(result),
        "qubits_measured": list(range(len(sample_bitstring))),
        "registers_mapping": {qi: qi for qi in range(num_random_unitary)},
        "actual_num_qubits": num_classical_register,
        "unitary_located": list(range(num_random_unitary)),
    }
    commons = {
        "exp_id": exp_id_process(None),
        "target_keys": [],
        "shots": 1,
        "backend": "qua_libs_transformed",
        "run_args": {},
        "transpile_args": {},
        "tags": tags,
        "save_location": Path(save_location) if save_location else Path("./"),
        "serial": None,
        "summoner_id": None,
        "summoner_name": None,
        "datetimes": {
            "transform-from-qua_libs": current_time(),
        },
    }
    outfields = {
        "denoted": "This is a QuaLibs result converted to Qurrium single-shot format.",
    }

    classical_shadow_exp = ShadowUnveilExperiment(
        arguments=args, commonparams=commons, outfields=outfields
    )

    classical_shadow_exp.beforewards.side_product["random_unitary_ids"] = {}
    for idx, (bitstring, gate_indices) in enumerate(result):
        classical_shadow_exp.afterwards.counts.append({bitstring: 1})
        classical_shadow_exp.beforewards.side_product["random_unitary_ids"][idx] = {
            qi: gate_idx for qi, gate_idx in enumerate(gate_indices)
        }

    return classical_shadow_exp

In [43]:
tranformed_exp_q1_delay_01 = qua_libs_result_to_qurrium_single_shot(qua_result_01)

In [44]:
tranformed_exp_q1_delay_01.analyze([0])

<ShadowUnveilAnalysis(
  serial=0,
  AnalysisInput(num_qubits=1, selected_qubits=[0], registers_mapping={0: 0}, bitstring_mapping={0: 0}, shots=1, unitary_located=[0]),
  AnalysisContent(purity=0.9918108108108108, entropy=0.011863143968181665, and others)),
  unused_args_num=1
  )>

In [45]:
tranformed_exp_q1_delay_01.statesheet()

 # ShadowUnveilExperiment with exp_id=a9d9c534-84d6-462e-b892-cda9f639be48
 - arguments
   - exp_name --------------------- qua_libs_experiment
   - times ------------------------ 1000
   - qubits_measured -------------- [0]
   - registers_mapping ------------ {0: 0}
   - actual_num_qubits ------------ 1
   - unitary_located -------------- [0]
   - random_unitary_seeds --------- None
 - commonparams
   - exp_id ----------------------- a9d9c534-84d6-462e-b892-cda9f639be48   # ()
   - target_keys ------------------ [] ...........................  # ()
   - shots ------------------------ 1 ............................  # ()
   - backend ---------------------- qua_libs_transformed .........  # ()
   - run_args --------------------- {} ...........................  # ()
   - transpile_args --------------- {} ...........................  # ()
   - tags ------------------------- () ...........................  # ()
   - save_location ---------------- . ............................  # ()
   - s

In [46]:
tranformed_exp_q1_delay_01.write()

('a9d9c534-84d6-462e-b892-cda9f639be48',
 {'folder': 'qua_libs_experiment.001',
  'qurryinfo': 'qua_libs_experiment.001/qurryinfo.json',
  'args': 'qua_libs_experiment.001/args/qua_libs_experiment.001.id=a9d9c534-84d6-462e-b892-cda9f639be48.args.json',
  'advent': 'qua_libs_experiment.001/advent/qua_libs_experiment.001.id=a9d9c534-84d6-462e-b892-cda9f639be48.advent.json',
  'legacy': 'qua_libs_experiment.001/legacy/qua_libs_experiment.001.id=a9d9c534-84d6-462e-b892-cda9f639be48.legacy.json',
  'tales.random_unitary_ids': 'qua_libs_experiment.001/tales/qua_libs_experiment.001.id=a9d9c534-84d6-462e-b892-cda9f639be48.random_unitary_ids.json',
  'reports': 'qua_libs_experiment.001/reports/qua_libs_experiment.001.id=a9d9c534-84d6-462e-b892-cda9f639be48.reports.json',
  'reports.tales.rho_m_dict': 'qua_libs_experiment.001/tales/qua_libs_experiment.001.id=a9d9c534-84d6-462e-b892-cda9f639be48.rho_m_dict.reports.json'})

In [47]:
main01_tranformed, side_product01_tranformed = tranformed_exp_q1_delay_01.reports[
    0
].export()
main01_tranformed

{'expect_rho': array([[0.005 +0.j    , 0.0315-0.0435j],
        [0.0315+0.0435j, 0.995 +0.j    ]]),
 'purity': np.float64(0.9918108108108108),
 'entropy': np.float64(0.011863143968181665),
 'classical_registers_actually': [0],
 'taking_time': 0.003976106643676758,
 'input': {'num_qubits': 1,
  'selected_qubits': [0],
  'registers_mapping': {0: 0},
  'bitstring_mapping': {0: 0},
  'shots': 1,
  'unitary_located': [0]},
 'header': {'serial': 0,
  'datetime': '2025-05-27 18:23:40',
  'summoner': None,
  'log': {}}}

In [48]:
exp_method04.exps[exp_q1_delay_01].analyze([0])

<ShadowUnveilAnalysis(
  serial=0,
  AnalysisInput(num_qubits=1, selected_qubits=[0], registers_mapping={0: 0}, bitstring_mapping={0: 0}, shots=1, unitary_located=[0]),
  AnalysisContent(purity=0.9918108108108108, entropy=0.011863143968181665, and others)),
  unused_args_num=1
  )>

In [49]:
main01, side_product01 = exp_method04.exps[exp_q1_delay_01].reports[0].export()
main01

{'expect_rho': array([[0.005 +0.j    , 0.0315-0.0435j],
        [0.0315+0.0435j, 0.995 +0.j    ]]),
 'purity': np.float64(0.9918108108108108),
 'entropy': np.float64(0.011863143968181665),
 'classical_registers_actually': [0],
 'taking_time': 0.0049860477447509766,
 'input': {'num_qubits': 1,
  'selected_qubits': [0],
  'registers_mapping': {0: 0},
  'bitstring_mapping': {0: 0},
  'shots': 1,
  'unitary_located': [0]},
 'header': {'serial': 0,
  'datetime': '2025-05-27 18:23:43',
  'summoner': None,
  'log': {}}}

In [50]:
exp_method04.exps[exp_q1_delay_01].statesheet()

 # ShadowUnveilExperiment with exp_id=1b224bc9-867f-495b-9404-b5c635cb0ecc
 - arguments
   - exp_name --------------------- experiment.N_U_1000.qurshady_entropy
   - times ------------------------ 1000
   - qubits_measured -------------- [0]
   - registers_mapping ------------ {0: 0}
   - actual_num_qubits ------------ 1
   - unitary_located -------------- [0]
   - random_unitary_seeds --------- None
 - commonparams
   - exp_id ----------------------- 1b224bc9-867f-495b-9404-b5c635cb0ecc ...  # ()
   - target_keys ------------------ ['q1_delay'] ...........................  # ()
   - shots ------------------------ 1 ......................................  # ()
   - backend ---------------------- <AerSimulator('aer_simulator')> ........  # ()
   - run_args --------------------- {} .....................................  # ()
   - transpile_args --------------- {} .....................................  # ()
   - tags ------------------------- () .....................................  # ()

In [51]:
exp_method04.exps[exp_q1_delay_01].write()

('1b224bc9-867f-495b-9404-b5c635cb0ecc',
 {'folder': 'experiment.N_U_1000.qurshady_entropy.001',
  'qurryinfo': 'experiment.N_U_1000.qurshady_entropy.001/qurryinfo.json',
  'args': 'experiment.N_U_1000.qurshady_entropy.001/args/experiment.N_U_1000.qurshady_entropy.001.id=1b224bc9-867f-495b-9404-b5c635cb0ecc.args.json',
  'advent': 'experiment.N_U_1000.qurshady_entropy.001/advent/experiment.N_U_1000.qurshady_entropy.001.id=1b224bc9-867f-495b-9404-b5c635cb0ecc.advent.json',
  'legacy': 'experiment.N_U_1000.qurshady_entropy.001/legacy/experiment.N_U_1000.qurshady_entropy.001.id=1b224bc9-867f-495b-9404-b5c635cb0ecc.legacy.json',
  'tales.random_unitary_ids': 'experiment.N_U_1000.qurshady_entropy.001/tales/experiment.N_U_1000.qurshady_entropy.001.id=1b224bc9-867f-495b-9404-b5c635cb0ecc.random_unitary_ids.json',
  'reports': 'experiment.N_U_1000.qurshady_entropy.001/reports/experiment.N_U_1000.qurshady_entropy.001.id=1b224bc9-867f-495b-9404-b5c635cb0ecc.reports.json',
  'reports.tales.rho_m_

## Recover Test

In [52]:
normal_01 = ShadowUnveilExperiment.read("experiment.N_U_1000.qurshady_entropy.001")
normal_01

| 0/1   0%|          | - Loading 1 experiments ... - 00:00 < ?

[<ShadowUnveilExperiment(exp_id=1b224bc9-867f-495b-9404-b5c635cb0ecc, 
   ShadowUnveilArguments(exp_name='experiment.N_U_1000.qurshady_entropy', times=1000, qubits_measured=[0], registers_mapping={0: 0}, actual_num_qubits=1, unitary_located=[0], random_unitary_seeds=None),
   Commonparams(exp_id='1b224bc9-867f-495b-9404-b5c635cb0ecc', target_keys=['q1_delay'], shots=1, backend='aer_simulator', run_args={}, transpile_args={}, tags=(), save_location='.', serial=None, summoner_id=None, summoner_name=None, datetimes=DatetimeDict({'build': '2025-05-27 18:23:19', 'run.001': '2025-05-27 18:23:19'})),
   unused_args_num=2,
   analysis_num=1))>]

In [54]:
normal_01[0].statesheet()

<ShadowUnveilExperiment(exp_id=1b224bc9-867f-495b-9404-b5c635cb0ecc, 
  ShadowUnveilArguments(exp_name='experiment.N_U_1000.qurshady_entropy', times=1000, qubits_measured=[0], registers_mapping={0: 0}, actual_num_qubits=1, unitary_located=[0], random_unitary_seeds=None),
  Commonparams(exp_id='1b224bc9-867f-495b-9404-b5c635cb0ecc', target_keys=['q1_delay'], shots=1, backend='aer_simulator', run_args={}, transpile_args={}, tags=(), save_location='.', serial=None, summoner_id=None, summoner_name=None, datetimes=DatetimeDict({'build': '2025-05-27 18:23:19', 'run.001': '2025-05-27 18:23:19'})),
  unused_args_num=2,
  analysis_num=1))>

In [57]:
normal_01[0].analyze([0])

<ShadowUnveilAnalysis(
  serial=1,
  AnalysisInput(num_qubits=1, selected_qubits=[0], registers_mapping={0: 0}, bitstring_mapping={0: 0}, shots=1, unitary_located=[0]),
  AnalysisContent(purity=0.9918108108108108, entropy=0.011863143968181665, and others)),
  unused_args_num=1
  )>

In [53]:
tranformed_01 = ShadowUnveilExperiment.read("qua_libs_experiment.001")
tranformed_01

| 0/1   0%|          | - Loading 1 experiments ... - 00:00 < ?

[<ShadowUnveilExperiment(exp_id=a9d9c534-84d6-462e-b892-cda9f639be48, 
   ShadowUnveilArguments(exp_name='qua_libs_experiment', times=1000, qubits_measured=[0], registers_mapping={0: 0}, actual_num_qubits=1, unitary_located=[0], random_unitary_seeds=None),
   Commonparams(exp_id='a9d9c534-84d6-462e-b892-cda9f639be48', target_keys=[], shots=1, backend='qua_libs_transformed', run_args={}, transpile_args={}, tags=(), save_location='.', serial=None, summoner_id=None, summoner_name=None, datetimes=DatetimeDict({'transform-from-qua_libs': '2025-05-27 18:23:40'})),
   unused_args_num=1,
   analysis_num=1))>]

In [55]:
tranformed_01[0].statesheet()

 # ShadowUnveilExperiment with exp_id=a9d9c534-84d6-462e-b892-cda9f639be48
 - arguments
   - exp_name --------------------- qua_libs_experiment
   - times ------------------------ 1000
   - qubits_measured -------------- [0]
   - registers_mapping ------------ {0: 0}
   - actual_num_qubits ------------ 1
   - unitary_located -------------- [0]
   - random_unitary_seeds --------- None
 - commonparams
   - exp_id ----------------------- a9d9c534-84d6-462e-b892-cda9f639be48   # ()
   - target_keys ------------------ [] ...........................  # ()
   - shots ------------------------ 1 ............................  # ()
   - backend ---------------------- qua_libs_transformed .........  # ()
   - run_args --------------------- {} ...........................  # ()
   - transpile_args --------------- {} ...........................  # ()
   - tags ------------------------- () ...........................  # ()
   - save_location ---------------- . ............................  # ()
   - s

In [56]:
tranformed_01[0].analyze([0])


<ShadowUnveilAnalysis(
  serial=1,
  AnalysisInput(num_qubits=1, selected_qubits=[0], registers_mapping={0: 0}, bitstring_mapping={0: 0}, shots=1, unitary_located=[0]),
  AnalysisContent(purity=0.9918108108108108, entropy=0.011863143968181665, and others)),
  unused_args_num=1
  )>